In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow import keras
from tensorflow.keras import layers
import ast  # Library for handling literal_eval
from geopy.distance import geodesic
from sklearn.ensemble import IsolationForest

# Load the dataset
df = pd.read_csv('merged.csv')
df = df.dropna()

# Print the column names to verify
print(df.columns)

# Setup the Isolation Forest
iso_forest = IsolationForest(n_estimators=500, contamination=0.01, random_state=42)
# Fit on the 'duration' column, you may want to reshape it if it's a Series
df['anomaly'] = iso_forest.fit_predict(df[['duration']])

# Filter out the outliers
df = df[df['anomaly'] == 1]  # Keep only normal data points
df.drop(columns='anomaly', inplace=True)  # Clean up by dropping the anomaly column

Index(['Main_Category', 'Longitude', 'Latitude', 'Day', 'duration',
       'Primary_Vehicle', 'Secondary_Vehicle', 'Is_Major_Incident', 'Advice_A',
       'Advice_B', 'Closure_Type', 'Direction', 'Main_Street',
       'Affected_Lanes', 'Actual_Number_of_Lanes ', 'Suburb', 'Traffic_Volume',
       'SA2_CODE21', 'SA3_CODE21', 'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21',
       'AREASQKM21', '0_ZID', '1_Area', '2_ML', '3_TRL', '4_PRL', '5_SRL',
       '6_TrRL', '7_RRL', '8_LsRL', '9_URL', '10_ToRL', '11_EoR', '12_NoN',
       '13_NDEs', '14_NNC2L', '15_NNC3L', '16_NNC4L', '17_AND', '18_NE',
       '19_MCI', '20_CoI', '21 NBS', '22_CA', '23_EA', '24_HA', '25_IA',
       '26_OA', '27_PA', '28_PrA', '29_RA', '30_TA', '31_WbA', '32_EoLU',
       '33_TP', '34_PD0MV', '35_PD1MV', '36_PD2MV', '37_PD3MV', '38_PD≥4MV',
       '39_PUE', '40_AMI', '41_NPTtWbyPT', '42_NPTtWbyTx', '43_NPTtWbyCD',
       '44_NPTtWbyCP', '45_NPTtWbyO', '46_NPWfH', '47_PWCJH', '48_PBCJH',
       '49_ANP_FH', 'RMS', 'Motorwa

C:\Python3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [2]:
import pandas as pd
from lifelines import LogLogisticAFTFitter
import matplotlib.pyplot as plt

df = df[df.duration>1]

aft = LogLogisticAFTFitter()
aft.fit(df, 'duration', event_col='Hour')

print(aft.summary)

C:\Python3\lib\site-packages\lifelines\fitters\__init__.py:2097: StatisticalWarning: The diagonal of the variance_matrix_ has negative values. This could be a problem with LogLogisticAFTFitter's fit to the data.

It's advisable to not trust the variances reported, and to be suspicious of the fitted parameters too.

  warnings.warn(warning_text, exceptions.StatisticalWarning)


                                  coef  exp(coef)   se(coef)  coef lower 95%   
param  covariate                                                               
alpha_ 0_ZID             -1.660854e-09   1.000000        NaN             NaN  \
       10_ToRL            9.381879e-05   1.000094  14.540815      -28.499379   
       11_EoR             3.499575e-02   1.035615   0.065299       -0.092987   
       12_NoN            -1.343522e-04   0.999866   0.004189       -0.008344   
       13_NDEs           -3.767904e-04   0.999623   0.004045       -0.008305   
...                                ...        ...        ...             ...   
       Traffic_Volume    -8.759689e-02   0.916130   0.002949       -0.093378   
       Transport for NSW -1.930375e-01   0.824451   0.007896       -0.208514   
       distance_to_CBD    6.108903e-03   1.006128   0.001241        0.003677   
       Intercept          3.346612e+00  28.406333        NaN             NaN   
beta_  Intercept          7.124133e-01  

In [4]:
aft.summary.round(3).to_csv('LogLogisticAFTFitter_Summary.csv')